In [46]:
from catboost import CatBoostClassifier, Pool
import pandas as pd
import os
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import glob
import os
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import optuna

In [47]:
# Функция для обучения на 1 наборе данных
def fitting(path):
    try:
        # Получим список файлов по выданному пути к папке
        current_data = os.listdir(path)
    # В случае ошибки отловим ее и прервем выполнение цикла
    except Exception:
        return "Не папка"
    # В случае отсутствия ошибки - сохраним данные
    else:
        current_data = os.listdir(path)
    # Выделим тренировочный датасет
    train_data = [data for data in current_data if data.endswith('train.parquet')][0]
    # Выделим тестовый датасет
    test_data = [data for data in current_data if data.endswith('test.parquet')][0]
    # Откроем тренировочные данные
    print("reading data", path)
    train_data = pd.read_parquet(path + f'/{train_data}')
    # train_data = train_data.iloc[:3000]
    # Откроем тестовые данные
    test_data = pd.read_parquet(path + f'/{test_data}')
    # test_data = test_data.iloc[:3000]

    # ----------------------------------------------
    # Разделение на признаки и целевую переменную
    X = train_data.drop(columns=['target', 'id', 'smpl'])
    y = train_data['target']

    # Отбор важных признаков с использованием GradientBoostingClassifier
    model = lgb.LGBMClassifier(random_state=10, n_jobs=16, verbose=-1)
    model.fit(X, y)

    # Селектор фич, у которых importance выше среднего значения
    selector = SelectFromModel(model, threshold="mean", prefit=True)
    X_selected = selector.transform(X)

    print("Количество выбранных признаков:", X_selected.shape[1])

    # Применение селектора к тестовому набору данных
    X_test = test_data.drop(columns=['id', 'smpl'])
    X_test_selected = selector.transform(X_test)

    # Находим выбранные признаки
    # Получаем маску выбранных признаков из селектора
    selected_features_mask = selector.get_support()

    # Извлекаем названия выбранных признаков, используя маску
    im_f = X.columns[selected_features_mask]
    im_f = im_f.tolist()

    print("Выбранные признаки:", im_f)

    X_selected = train_data[im_f].copy()
    X_test_selected = test_data[im_f].copy()
    y = train_data['target']

    # Обучаем LightGBM с кросс-валидацией на выбранных признаках
    lgb_model = lgb.LGBMClassifier(verbose=-1)
    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=10)  # стратификация по таргету
    scores = cross_val_score(lgb_model, X_selected, y, cv=skf, scoring='roc_auc')
    print("ROC-AUC на LightGBM с кросс-валидацией:", scores.mean())
    
    # Определяем функцию для оптимизации гиперпараметров
    def objective(trial):
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.15),
            'max_depth': trial.suggest_int('max_depth', 3, 13),
            'num_leaves': trial.suggest_int('num_leaves', 50, 1000),
            'n_estimators': trial.suggest_int('n_estimators', 200, 700),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 90),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'lambda_l1': trial.suggest_float('lambda_l1', 1e-6, 1.0, log = True),
            'lambda_l2': trial.suggest_float('lambda_l2', 1e-6, 1.0, log = True),
        }
        model = lgb.LGBMClassifier(**param, verbose=-1, n_jobs=16)
        scores = cross_val_score(model, X_selected, y, cv=skf, scoring='roc_auc')
        return scores.mean()

    # Запускаем Optuna для оптимизации
    study = optuna.create_study(direction="maximize")
    # Warm-start
    warm_start_small = {
        'learning_rate': 0.01290806719885743, 
        'max_depth': 3, 
        'num_leaves': 156, 
        'n_estimators': 500, 
        'min_child_samples': 32, 
        'subsample': 0.9052108095701373, 
        'colsample_bytree': 0.9153415375057271, 
        'lambda_l1': 0.5216384377631789, 
        'lambda_l2': 0.5141215730887658,
    }
    
    warm_start_large = {
        'learning_rate': 0.005, 
        'max_depth': -1, 
        'num_leaves': 512, 
        'n_estimators': 450, 
        'min_child_samples': 32, 
        'subsample': 0.7, 
        'colsample_bytree': 0.7, 
        'lambda_l1': 1, 
        'lambda_l2': 1,
    }
    
    warm_start_middle = {
        'learning_rate': 0.04, 
        'max_depth': -1, 
        'num_leaves': 128, 
        'n_estimators': 400, 
        'min_child_samples': 20, 
        'subsample': 0.65, 
        'colsample_bytree': 0.9, 
        'lambda_l1': 1, 
        'lambda_l2': 0,
    }
    
    num_rows = train_data.shape[0]
    num_iterations = 50
    warm_start = {}
    if num_rows < 70000:
        num_iterations = 30
        warm_start = warm_start_small
    elif num_rows < 300000:
        num_iterations = 20
        warm_start = warm_start_middle
    else:
        num_iterations = 10
        warm_start = warm_start_large
        
    
    study.enqueue_trial({**warm_start})
    study.optimize(objective, n_trials=num_iterations, timeout=1200)

    # Получаем лучшие параметры
    best_params = study.best_params
    print("Лучшие параметры:", best_params)

    # Финальное обучение модели с лучшими параметрами
    final_model = lgb.LGBMClassifier(**best_params, n_jobs=16, verbose = -1)
    final_model.fit(X_selected, y)

    # Предсказания для тестовых данных
    y_test_pred = final_model.predict_proba(X_test_selected)[:, 1]
    
    # ---------------------------------------
    # Объединим предсказание с метками
    test_data['target'] = y_test_pred
    # Отсортируем предсказание
    prediction = test_data[['id', 'target']].sort_values(by='id', ascending=True)
    # Вернем предсказание, как результат работы модели
    return prediction

In [48]:
# Функция смотрит на рабочее окружение и на папки с данными
def model():
    # Пропишем путь к файлам данных
    data = 'data'
    # Запишем список датасетов в папке:
    folders = os.listdir(data)
    # Создадим цикл для прохождения по каждому файлу и генерации предсказания
    for fold in folders:
        print("Training on", fold)
        # Запишем новый путь к данным
        data_path = data + f'/{fold}'
        # Вызовем функцию, передав в нее путь к папке для обучения
        prediction = fitting(path=data_path)
        # Сохраним полученное предсказание
        if type(prediction) is not str:
            # Сохраняем предсказание
            prediction.to_csv(f"predictions/{fold}.csv", index=False)
            print("Предсказание создано!")
        else:
            print("Невозможно создать предсказание!")
    
    '''print("Training on pd_ul_9")
    # Запишем новый путь к данным
    data_path = r"C:\Users\79168\Desktop\FINAL\pd_ul_9"
    # Вызовем функцию, передав в нее путь к папке для обучения
    prediction = fitting(path=data_path)
    # Сохраним полученное предсказание
    if type(prediction) is not str:
        # Сохраняем предсказание
        prediction.to_csv(r"C:\Users\79168\Desktop\FINAL\predictions\pd_ul_9.csv", index=False)
        print("Предсказание создано!")
    else:
        print("Невозможно создать предсказание!")'''

In [49]:
# Обозначаем действия при запуске кода
if __name__ == "__main__":
    # Запускаем модель
    model()

Training on pd_ul_9
reading data C:\Users\79168\Desktop\FINAL\pd_ul_9


C:\Users\79168\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\79168\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Количество выбранных признаков: 109
Выбранные признаки: ['feature_1', 'feature_2', 'feature_4', 'feature_6', 'feature_8', 'feature_13', 'feature_18', 'feature_22', 'feature_27', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_39', 'feature_47', 'feature_50', 'feature_51', 'feature_53', 'feature_56', 'feature_57', 'feature_67', 'feature_72', 'feature_74', 'feature_75', 'feature_78', 'feature_86', 'feature_87', 'feature_90', 'feature_93', 'feature_99', 'feature_102', 'feature_111', 'feature_114', 'feature_115', 'feature_118', 'feature_123', 'feature_125', 'feature_129', 'feature_130', 'feature_132', 'feature_140', 'feature_143', 'feature_144', 'feature_146', 'feature_149', 'feature_150', 'feature_154', 'feature_156', 'feature_158', 'feature_167', 'feature_169', 'feature_173', 'feature_177', 'feature_178', 'feature_182', 'feature_183', 'feature_184', 'feature_186', 'feature_188', 'feature_192', 'feature_194', 'feature_207', 'feature_208', 'feat

[I 2024-12-07 18:37:23,087] A new study created in memory with name: no-name-4704d372-c210-4104-a12e-c595b0e6b0a8


ROC-AUC на LightGBM с кросс-валидацией: 0.7103090438053156
[LightGBM] [Warning] lambda_l1 is set=0.5216384377631789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5216384377631789
[LightGBM] [Warning] lambda_l2 is set=0.5141215730887658, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5141215730887658
[LightGBM] [Warning] lambda_l1 is set=0.5216384377631789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5216384377631789
[LightGBM] [Warning] lambda_l2 is set=0.5141215730887658, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5141215730887658
[LightGBM] [Warning] lambda_l1 is set=0.5216384377631789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5216384377631789
[LightGBM] [Warning] lambda_l2 is set=0.5141215730887658, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5141215730887658
[LightGBM] [Warning] lambda_l1 is set=0.5216384377631789, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5216384377631789
[LightG

...

[LightGBM] [Warning] lambda_l1 is set=0.00016723466011908268, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00016723466011908268
[LightGBM] [Warning] lambda_l2 is set=0.13270460315610808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.13270460315610808
[LightGBM] [Warning] lambda_l1 is set=0.00016723466011908268, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00016723466011908268
[LightGBM] [Warning] lambda_l2 is set=0.13270460315610808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.13270460315610808
[LightGBM] [Warning] lambda_l1 is set=0.00016723466011908268, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00016723466011908268
[LightGBM] [Warning] lambda_l2 is set=0.13270460315610808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.13270460315610808
[LightGBM] [Warning] lambda_l1 is set=0.00016723466011908268, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00016723466011908268
[LightGBM] [Warning] lambda_

[I 2024-12-07 18:39:09,609] Trial 28 finished with value: 0.8204554122118332 and parameters: {'learning_rate': 0.08787989666226396, 'max_depth': 7, 'num_leaves': 975, 'n_estimators': 573, 'min_child_samples': 60, 'subsample': 0.7958063401217346, 'colsample_bytree': 0.6514987002703406, 'lambda_l1': 0.00016723466011908268, 'lambda_l2': 0.13270460315610808}. Best is trial 14 with value: 0.8473030338896123.


[LightGBM] [Warning] lambda_l1 is set=0.00031166314116054674, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00031166314116054674
[LightGBM] [Warning] lambda_l2 is set=0.6011004821628853, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6011004821628853
[LightGBM] [Warning] lambda_l1 is set=0.00031166314116054674, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00031166314116054674
[LightGBM] [Warning] lambda_l2 is set=0.6011004821628853, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6011004821628853
[LightGBM] [Warning] lambda_l1 is set=0.00031166314116054674, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00031166314116054674
[LightGBM] [Warning] lambda_l2 is set=0.6011004821628853, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6011004821628853
[LightGBM] [Warning] lambda_l1 is set=0.00031166314116054674, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00031166314116054674
[LightGBM] [Warning] lambda_l2 is 

[I 2024-12-07 18:39:12,844] Trial 29 finished with value: 0.8080586788706093 and parameters: {'learning_rate': 0.10563327401816913, 'max_depth': 3, 'num_leaves': 285, 'n_estimators': 489, 'min_child_samples': 55, 'subsample': 0.8608591618486459, 'colsample_bytree': 0.552672815109823, 'lambda_l1': 0.00031166314116054674, 'lambda_l2': 0.6011004821628853}. Best is trial 14 with value: 0.8473030338896123.


Лучшие параметры: {'learning_rate': 0.10696883335343621, 'max_depth': 4, 'num_leaves': 933, 'n_estimators': 609, 'min_child_samples': 73, 'subsample': 0.8263374541164522, 'colsample_bytree': 0.5271035377802693, 'lambda_l1': 0.043995181994689214, 'lambda_l2': 0.08959455853014209}
[LightGBM] [Warning] lambda_l1 is set=0.043995181994689214, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.043995181994689214
[LightGBM] [Warning] lambda_l2 is set=0.08959455853014209, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.08959455853014209
Предсказание создано!
